In [239]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [240]:
file = 'yandex_hack.xlsx'
df = pd.read_excel(file)
df.order_created_datetime = pd.to_datetime(train_df.order_created_datetime)
train_df = df[df.brand_name=='A']
test_df = df[df.brand_name=='B']

Определимся, как далеко друг от друга по времени комбинрованные заказы 

In [242]:
seconds = []
for i,row in train_df.iterrows():
    if row['batched_with_order_id'] is not np.nan:
        time1 = train_df.query("order_id == '{}'".format(row['batched_with_order_id'])).order_created_datetime.item()
        time2 = row['order_created_datetime']
        seconds.append(abs((time2-time1).total_seconds()))
sec = pd.DataFrame(seconds, columns = ['delta'])
sec.describe()
sec.quantile(.95)

delta    1028.0
Name: 0.95, dtype: float64

Посчитаем, как много заказов попадают в данный промежуток времени при условии, что ресторан и город, а также курьер - одинаковые.

In [237]:
ids = []
skip_ids = []
total = 0

for i,row in train_df.iterrows():
    if row['order_id'] not in skip_ids: 
        from_time = row['order_created_datetime']
        to_time = row['order_created_datetime']+dt.timedelta(seconds=1028)#579
        condition = '''
        city == '{}' and rest_id == '{}' and order_created_datetime > '{}' and order_created_datetime < '{}' and courier_id == '{}' and order_id != '{}'
        '''.format(row['city'], row['rest_id'], from_time, to_time, row['courier_id'], row['order_id'])
        pretendents = train_df.query(condition)
        if len(pretendents) >=1 and len(pretendents)<=2:
            total += 1
            skip_ids.append(row['order_id'])
print(total)




707


Получили число, больше, реальность. Осталось догадаться, как отфильтровать полученные значения.
Далее считаем, сколько из полученной суммы - это правильно угаданные ответы.

In [238]:
train_df[train_df.order_id.isin(skip_ids)].batched_with_order_id.value_counts().sum()

209

Получили меньше, чем нужно, но, если умножить на 2 (поскольку для каждой пары мы находим инкрементируем наше значение только на один), то мы нашли  почти всех, но, к сожалению, ошибочные ответы тоже нужно умножить на 2.